# Imports

In [1]:
import mimicLOB as lob
from mimicLOB.agent.randomAgent import randomAgent
from apscheduler.schedulers.background import BackgroundScheduler

import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sortedcontainers import SortedDict
from pprint import pprint

# local server
localserver = "http://127.0.0.1:5001"

# distant server
distantserver = "http://mimiclob.appspot.com"

# Server used
server = localserver

# Executed Trades
executedtrades = SortedDict()

# pending Sent Orders 
pendingtrades = SortedDict()


# Get the LOB state

In [2]:
response = requests.get(f"{server}/getLOBstate")
tickSize =  requests.get(f"{server}/getticksize").json()['ticksize']

try:
    LOBstate = pd.read_json(response.json()['LOBstate'])
    LOBstate.sort_index(inplace=True)
    minPrice = LOBstate.index[0]
    maxPrice = LOBstate.index[-1]
    LOBstate = LOBstate.reindex(range(minPrice, maxPrice, tickSize))
    LOBstate.columns = ['quantities']
    print(LOBstate)
    LOBstate.plot.bar(figsize=(20, 7))
except:
    print('LOB is empty')

LOB is empty


## Simulation configuration

In [3]:
config =   {'agents'      : {'agent1' : {'type'         : 'randomAgent',
                                        'subtype'       : 'randomLimitBuyer',
                                        'server'        : server,
                                        'intensity'     : 0.4,
                                        'quantityRange' : [50, 200, 10]},
                            'agent2' : {'type'          : 'randomAgent',
                                        'subtype'       : 'randomMarketBuyer',
                                        'server'        : server,
                                        'intensity'     : 0.4,
                                        'quantityRange' : [50, 200, 10]},
                            'agent3' : {'type'          : 'randomAgent',
                                        'subtype'       : 'randomLimitSeller',
                                        'server'        : server,
                                        'intensity'     : 0.4,
                                        'quantityRange' : [50, 200, 10]},
                            'agent4' : {'type'          : 'randomAgent',
                                        'subtype'       : 'randomMarketSeller',
                                        'server'        : server,
                                        'intensity'     : 0.4,
                                        'quantityRange' : [50, 200, 10]},
                            'MM'     : {'type'          : 'basicMM',
                                        'server'        : server,
                                        'refQuantity'   : 100,
                                        'refPrice'      : 100}}}

## Create the agents

In [4]:
agentFactory = {} # keeps references on agents
config_agents = config['agents']

def getInstance(module, defObject, **kwargs):
    typ = defObject['type']
    exec(f'global myModule; from {module} import {typ} as myModule')
    class_ = getattr(myModule, typ)
    return class_(**defObject)

for agent in config_agents:
    agentFactory[agent] = getInstance('mimicLOB.agent', config_agents[agent])        

## Launch News

In [5]:
# start news
response = requests.get(f"{server}/launchNews").json()
response['status']

'DONE'

## Launch Trading

In [6]:
sched = BackgroundScheduler()
sched.start()

def startSimulation():
    for agent in agentFactory:
        agentFactory[agent].jobSO = sched.add_job(agentFactory[agent].sendOrders, 'interval', seconds=0.1, jitter=0.1)
        agentFactory[agent].JobCO = sched.add_job(agentFactory[agent].cancelFarAwayOrders, 'interval', seconds=1, jitter=0.5) 

def stopSimulation():
    for agent in agentFactory:
        if agentFactory[agent].JobSO is not None:  agentFactory[agent].JobSO.remove()       
        if agentFactory[agent].JobCO is not None:  agentFactory[agent].JobCO.remove()    
        agentFactory[agent].JobSO = None
        agentFactory[agent].JobCO = None

In [14]:
startSimulation()

## Pause Trading

In [13]:
try:
    sched.pause()
except Exception as e:
    print(e)

## Resume Trading

In [9]:
try:
    sched.resume()
except Exception as e:
    print(e)

Run time of job "randomAgent.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:05:44 CEST)" was missed by 0:00:01.023536
Run time of job "randomAgent.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:05:44 CEST)" was missed by 0:00:01.290095
Run time of job "randomAgent.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:05:44 CEST)" was missed by 0:00:01.661291
Run time of job "randomAgent.cancelFarAwayOrders (trigger: interval[0:00:01], next run at: 2020-04-08 10:05:44 CEST)" was missed by 0:00:02.505418
Run time of job "randomAgent.cancelFarAwayOrders (trigger: interval[0:00:01], next run at: 2020-04-08 10:05:45 CEST)" was missed by 0:00:02.296525
Run time of job "basicMM.cancelFarAwayOrders (trigger: interval[0:00:01], next run at: 2020-04-08 10:05:45 CEST)" was missed by 0:00:02.160904
Run time of job "randomAgent.cancelFarAwayOrders (trigger: interval[0:00:01], next run at: 2020-04-08 10:05:44 CEST)" was missed

ERROR canceling far away orders float() argument must be a string or a number, not 'NoneType'
ERROR canceling far away orders unsupported operand type(s) for -: 'NoneType' and 'int'


Execution of job "randomAgent.cancelFarAwayOrders (trigger: interval[0:00:01], next run at: 2020-04-08 10:05:52 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "randomAgent.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:05:53 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "randomAgent.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:05:53 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "basicMM.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:05:53 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "randomAgent.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:05:53 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "basicMM.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:05:53 CEST)" skipped: maximum nu

ERROR canceling far away orders float() argument must be a string or a number, not 'NoneType'
ERROR canceling far away orders unsupported operand type(s) for -: 'NoneType' and 'int'


Execution of job "randomAgent.cancelFarAwayOrders (trigger: interval[0:00:01], next run at: 2020-04-08 10:05:55 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "randomAgent.cancelFarAwayOrders (trigger: interval[0:00:01], next run at: 2020-04-08 10:05:55 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "randomAgent.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:05:55 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "basicMM.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:05:56 CEST)" skipped: maximum number of running instances reached (1)


...

Execution of job "randomAgent.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:07:05 CEST)" skipped: maximum number of running instances reached (1)


ERROR canceling far away orders float() argument must be a string or a number, not 'NoneType'


Execution of job "randomAgent.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:07:06 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "randomAgent.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:07:07 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "randomAgent.cancelFarAwayOrders (trigger: interval[0:00:01], next run at: 2020-04-08 10:07:07 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "randomAgent.cancelFarAwayOrders (trigger: interval[0:00:01], next run at: 2020-04-08 10:07:08 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "basicMM.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:07:08 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "randomAgent.cancelFarAwayOrders (trigger: interval[0:00:01], next run at: 2020-04-08 10:07:08 CEST)" skipped: ma

ERROR canceling far away orders unsupported operand type(s) for -: 'NoneType' and 'int'
ERROR canceling far away orders float() argument must be a string or a number, not 'NoneType'


Execution of job "randomAgent.cancelFarAwayOrders (trigger: interval[0:00:01], next run at: 2020-04-08 10:07:11 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "basicMM.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:07:12 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "randomAgent.cancelFarAwayOrders (trigger: interval[0:00:01], next run at: 2020-04-08 10:07:12 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "basicMM.cancelFarAwayOrders (trigger: interval[0:00:01], next run at: 2020-04-08 10:07:12 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "randomAgent.sendOrders (trigger: interval[0:00:00.100000], next run at: 2020-04-08 10:07:13 CEST)" skipped: maximum number of running instances reached (1)
Execution of job "randomAgent.cancelFarAwayOrders (trigger: interval[0:00:01], next run at: 2020-04-08 10:07:13 CEST)" skipped: maxi

## Stop Trading & reset LOB

In [ ]:
try:
    sched.shutdown()
except Exception as e:
    print(e)

In [ ]:
response = requests.get(f"{server}/resetLOB").json()
response['status']

## Get Price Tape

In [ ]:
histoPrices = requests.get(f"{server}/getPriceTape",).json()['PriceTape']
histoPrices = pd.DataFrame([histoPrices]).T; histoPrices.columns = ['Prices']
histoPrices.plot(figsize=(20,7))
# display(histoPrices.describe())
display(f'open : {histoPrices.iloc[0,0]}')
display(f'high : {histoPrices.max()[0]}')
display(f'open : {histoPrices.min()[0]}')
display(f'close : {histoPrices.iloc[-1, 0]}')
plt.show()

## Get Transaction Tape

In [ ]:
TransactionTape = requests.get(f"{server}/getTransactionTape").json()['TransactionTape']
TransactionTape = pd.read_json(TransactionTape)

In [ ]:
TransactionTape